# Fonctions annexes

Commençons par coder les fonctions annexes. 

- degre(G) donnant le degré de chaque sommet
- trieSommets(G) triant les sommets suivant leur degré

On rappelle ici que le graphe est implémenté sous forme de liste d'adjacence. Ainsi le degré de chaque sommet est juste le nombre de ses voisins (le graphe n'est pas orienté, donc pas de degré sortant ou entrant). 

Pour le tri : on utilise la méthode <code>sort</code> de l'objet <code>List</code>. On peut alors utiliser l'argument optionnel <code>key=fonction servant à trier</code> pour spécifier le critère sur lequel on trie les sommets.  

In [148]:
def degre(G):
    return [len(G[k]) for k in range(len(G))]

In [149]:
def trieSommets(G):
    deg = degre(G)  # calcul du degré de chaque sommet
    sommets = [k for k in range(len(G))] # liste des sommets
    sommets.sort(key = lambda s: deg[s], reverse = True)
    return sommets

On peut tester notre fonction sur le graphe suivant : 

<img src="test.png">

In [150]:
G = [[1,2,3], [0,4], [0,4,5], [0,4], [1,2,3,5], [2,4]]
degre(G)

[3, 2, 3, 2, 4, 2]

In [151]:
L = trieSommets(G)

# Algorithme de Welsh-Powel

On rappelle ici le principe de l'algorithme : 

- Trier les sommets par ordre d ́ecroissant de degré pour obtenir la liste <code>sommetsAColorier</code>
- Tant que la liste <code>sommetsAColorier</code> n’est pas vide :
    - Prendre le premier sommet <code>sMax</code> de la liste et le supprimer.
    - Parcourir le reste de la liste au fur et à mesure et réunir les sommets de la liste qui ne sont pas adjacents aux sommets déjà réunis à <code>sMax</code>

Tout d'abord définissons comment nous allons implémenter les coloriage. Le nom des couleurs n'est pas intéressant en ce qui nous concerne, ce qui est important c'est de savoir les groupes de sommets qui sont de la même couleur. Il s'agit donc de **partitionner** les sommets en groupes de couleurs différentes : la structure Union-Find s'impose. 

Ensuite, la principale difficulté réside dans le fait qu'il ne faut pas colorier deux sommets adjacents de la même couleur. Ainsi, quand on va parcourir la liste des sommets à colorier, il faut savoir quels sont les sommets adjacents aux sommets qui sont de la même couleur que <code>sMax</code>. On stockera ces sommets dans une liste <code>sommetsAdjacents</code>. 

Commençons par définir les fonctions utiles à la manipulation d'une structure Union-Find. 

In [152]:
def racine(F,s):
    if F[s]==s:
        return s
    else:
        r=racine(F,F[s])  # optimisation, les hauteurs sont plus petites
        F[s]=r
        return r

def reunir(F,s1,s2):
    F[racine(F,s1)]=racine(F,s2)

def sontEquivalent(F,s1,s2):
    return racine(F,s1)==racine(F,s2)

On peut alors implémenter l'algorigthme : 

In [153]:
def colorationWP(G):
    # initialisation des sommets : on trie les sommets par degré décroissant
    sommetsAColorier = trieSommets(G) 
    # initialisation de la structure union-find
    coloration = [k for k in range(len(G))]

    while sommetsAColorier!=[]: # tant qu'il reste des sommets à colorier
        sMax = sommetsAColorier.pop(0) # on prend le premier sommet à colorier (degré max donc)
        sommetsAdjacents = G[sMax] # liste qui contient les sommets adjacents aux sommets de la même couleur que sMax (pour l'instant il n'y a que sMax)
        restants = [] # liste qui contiendra les sommets qui resteront à colorier
        for s in sommetsAColorier: # pour chaque sommet à colorier
            if s not in sommetsAdjacents: # s'il n'est pas adjacent à un sommet colorié de la même couleur que sMax
                reunir(coloration, s,sMax) # on le colorie de la même couleur que sMax
                sommetsAdjacents+=G[s] # on rajoute ses voisins à la liste des sommets adjacents (pour qu'ils ne soient pas de la même couleur)
            else:
                restants.append(s) # sinon, il va falloir le colorier plus tard
        sommetsAColorier = restants # on met à jour la liste des sommets à colorier
    return coloration # on renvoie la structure union-find correspondant à la coloration


On peut tester l'algorithme : 

In [154]:
colorationWP(G)

[4, 2, 2, 2, 4, 5]

Pour visualiser le résultat, on peut utiliser la fonction définie en cours : 

In [155]:
def affichePartition(F):
	N = len(F)
	dico = {}
	for s in range(N):
		r = racine(F,s)
		if r in dico:
			dico[r].append(s)
		else:
			dico[r]=[s]
	return [dico[r] for r in dico]

In [156]:
affichePartition(colorationWP(G))

[[0, 4], [1, 2, 3], [5]]

# Bonus : Arbre couvrant minimal

On rappelle ici l'algorithme : 

- *Initialisation*
    - on initialise un graphe arbre sans ar`etes
    - on selectionne un sommet depart (prendre 0) et on initialise une liste listeAretes constituée des arêtes issues de ce sommet.
- Tant que le liste listeAretes n’est pas vide :
    - On s ́electionne une arète aMin de poids minimal dans la liste listeAretes
    - On le supprime de la liste listeAretes
    - Si aMin=(s,t) avec s déjà relié à depart dans arbre et t non relié, alors on rajoute l’arc aMin au graphe arbre.
    - On rajoute les arêtes « utiles » issues de t à la liste listeAretes (c’est–à–dire les arêtes reliant deux sommets non déjà reliés).

On commence par les routines utiles : 

- <code>aretesIssuesDe(s,G)</code> renvoie les arêtes issues de s
- <code>areteMin(listeAretes)</code> renvoie l'arêtes minimale d'une slite d'arêtes

In [157]:
def aretesIssuesDe(s,G):
	listeAretes = []
	for i in range(len(G)):
		if G[s][i] != 0:
			listeAretes.append((s,G[s][i],i))
	return listeAretes

def areteMin(listeAretes):
	if listeAretes==[]:
		return None
	a = listeAretes[0]
	for (s,p,i) in listeAretes:
		if p<a[1]:
			a = (s,p,i)
	return a

On peut alors implémenter l'algorithme : 

In [158]:
def arbreCouvrantMin(G):
	arbre = [[0]*len(G) for k in range(len(G))] # on initialise un graphe sans arête
	listeAretes = aretesIssuesDe(0,G) # on initialise (en partant de 0) la liste des arêtes que l'on va éventuellement ajouter 
	composantes = [k for k in range(len(G))] # on initialise les composantes connexes de arbre (par d'arêtes donc que des sommets isolés)
	while listeAretes!=[]: # tant qu'il y a des arêtes à visiter
		(s,p,i) = areteMin(listeAretes) # on choisit l'ar^te de poids minimum
		listeAretes.remove((s,p,i)) # que l'on supprime de la liste
		if not sontEquivalent(composantes, s, i): # si c'est une arêts connectant deux sommets non déjà reliés
			arbre[s][i]=p # on l'ajoute à l'arbre
			arbre[i][s]=p # arbre est non orienté
			reunir(composantes, s, i) # on adapte les composantes connexes de l'arbre
			nouvellesAretesPossibles = aretesIssuesDe(i,G) # on considère les arêtes issues de i
			for (u,pp,v) in nouvellesAretesPossibles: # et un ne garde que celles reliant des sommets non déjà reliés
				if not sontEquivalent(composantes,u,v):
					listeAretes.append((u,pp,v))
	return arbre

In [159]:
# test
G = [
[0,4,2,3,0],
[4,0,5,0,2],
[2,5,0,-1,1],
[3,0,-1,0,0],
[0,2,1,0,0]
]

print(arbreCouvrantMin(G))

[[0, 0, 2, 0, 0], [0, 0, 0, 0, 2], [2, 0, 0, -1, 1], [0, 0, -1, 0, 0], [0, 2, 1, 0, 0]]
